Aby ste mohli spustiť nasledujúce poznámkové bloky, ak ste to ešte neurobili, musíte nastaviť openai kľúč v súbore .env ako `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Ďalej načítame index vektorových reprezentácií (Embedding Index) do Pandas DataFrame. Index vektorových reprezentácií je uložený v JSON súbore s názvom `embedding_index_3m.json`. Index vektorových reprezentácií obsahuje vektorové reprezentácie pre každý z prepisov z YouTube až do konca októbra 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Ďalej vytvoríme funkciu s názvom `get_videos`, ktorá vyhľadá v Embedding Indexe podľa zadaného dopytu. Funkcia vráti 5 videí, ktoré sú najviac podobné zadanému dopytu. Funkcia funguje nasledovne:

1. Najskôr sa vytvorí kópia Embedding Indexu.
2. Potom sa pre dopyt vypočíta Embedding pomocou OpenAI Embedding API.
3. Následne sa v Embedding Indexe vytvorí nový stĺpec s názvom `similarity`. Stĺpec `similarity` obsahuje kosínusovú podobnosť medzi Embeddingom dopytu a Embeddingom pre každý segment videa.
4. Potom sa Embedding Index filtruje podľa stĺpca `similarity`. Do výsledku sa zahrnú len videá, ktoré majú kosínusovú podobnosť väčšiu alebo rovnú 0,75.
5. Nakoniec sa Embedding Index zoradí podľa stĺpca `similarity` a vráti sa 5 najlepších videí.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Táto funkcia je veľmi jednoduchá, iba vypíše výsledky vyhľadávacieho dopytu.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Najprv sa Embedding Index načíta do Pandas Dataframe.
2. Následne je používateľ vyzvaný, aby zadal dopyt.
3. Potom sa zavolá funkcia `get_videos`, ktorá vyhľadá dopyt v Embedding Indexe.
4. Nakoniec sa zavolá funkcia `display_results`, ktorá zobrazí výsledky používateľovi.
5. Používateľ je potom opäť vyzvaný na zadanie ďalšieho dopytu. Tento proces sa opakuje, kým používateľ nezadá `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.sk.png)

Budete vyzvaní na zadanie dopytu. Zadajte dopyt a stlačte enter. Aplikácia vráti zoznam videí, ktoré sú relevantné k zadanému dopytu. Aplikácia tiež poskytne odkaz na miesto vo videu, kde sa nachádza odpoveď na vašu otázku.

Tu je niekoľko príkladov dopytov, ktoré môžete vyskúšať:

- Čo je Azure Machine Learning?
- Ako fungujú konvolučné neurónové siete?
- Čo je neurónová sieť?
- Môžem používať Jupyter Notebooks s Azure Machine Learning?
- Čo je ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Vyhlásenie o vylúčení zodpovednosti**:  
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Za autoritatívny zdroj sa považuje pôvodný dokument v jeho natívnom jazyku. Pre kritické informácie odporúčame profesionálny ľudský preklad. Nezodpovedáme za žiadne nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
